## Langgraph Essay Writer Agent 

You are an expert writer tasked with writing a high level outline of an essay.  Write such an outline for user provided topic.  Give an outline of the essay along with notes and instructions for the sections

##### Note: Go to JupyterLab terminal and execute following command before getting started
<pre>
    uv add langgraph
</pre>

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Optional
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from pydantic import BaseModel, ValidationError
#from langchain_core.pydantic_v1 import BaseModel
from langchain.tools import tool
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langgraph.checkpoint.sqlite import SqliteSaver
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Image
import re
import json


# connect to tavily search tool - use your tavily api key
os.environ['TAVILY_API_KEY']="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"
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
#tool = TavilySearchResults(max_results=2)

#define agent state
class AgentState(TypedDict):
    task: str
    lnode: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    queries: List[str]
    revision_number: int
    max_revisions: int
    count: Annotated[int, operator.add]


#define and configure the model
# configure model
httpx_client = httpx.Client(http2=True, verify=False, timeout=10.0)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]

model = ChatOpenAI(temperature=0, model=chat_credentials["model_name"], base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=httpx_client)


#define prompts
PLAN_PROMPT = """
You are an expert writer tasked with writing a high level outline of an eassy. \
Write such an outline for the user provided topic. Give an outline of eassy along \
with any relevant notes or instructions for the sections.
"""

WRITER_PROMPT = """
You are an eassy assistant tasked with writing excellent 5-paragraph eassys. \
Generate the best eassy possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of ypur previous attempts. \

--------

{content}"""

REFLECTION_PROMPT = """
You are a teacher grading an eassy submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc.
"""

RESEARCH_PLAN_PROMPT = """
You are a researcher charged with providing information that can \
be used when writing the following eassy. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max
"""

RESEARCH_CRITIQUE_PROMPT = """
You are a researcher charged with providing information that can \
be used when making any requested revisions (as oulined below). \
Generate a list of search queries that will gather any relevant information. \
Only generate 3 queries max.
"""
    
def extract_json(text):
    # Remove unwanted tags like <think> and <speak>
    cleaned_text = re.sub(r'<\/?[\w\d]+>', '', text).strip()

    # Now try to extract the JSON part using regex
    match = re.search(r'\{.*\}', cleaned_text, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found in response")
    return json.loads(match.group(0))

def normalize_to_queries(output: str):
    """
    Convert model output into a dict matching Queries schema.
    """
    # Remove <think>...</think> if present
    output = re.sub(r"<think>.*?</think>", "", output, flags=re.DOTALL).strip()

    # Try strict JSON parse
    try:
        return json.loads(output)
    except json.JSONDecodeError:
        # Fallback: convert markdown/bullets/numbered list into dict
        lines = [
            re.sub(r'^\s*[\d\-\*\.\)]*\s*', '', line).strip(' *"')
            for line in output.splitlines()
            if line.strip()
        ]
        return {"queries": lines}
    
class Queries(BaseModel):
    queries: List[str]

#implement nodes
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state["task"])
    ]
    response = model.invoke(messages)
    response_content = response.content if hasattr(response, 'content') else str(response)

    # Remove <think>...</think> blocks completely
    response_content = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()
    return {"plan": response_content}
    
def research_plan_node(state: AgentState):
    print("entering research_plan_node")
    raw_response = model.invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    response_content = raw_response.content if hasattr(raw_response, 'content') else str(raw_response)

    # Remove <think>...</think> blocks completely
    response_content = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()

    try:
        # Try parsing as JSON first
        try:
            json_data = json.loads(response_content)
        except json.JSONDecodeError:
            # Fallback: convert numbered/bulleted list into dict
            lines = [line.strip("0123456789. -") for line in response_content.splitlines() if line.strip()]
            json_data = {"queries": lines}

        print("json data in research plan mode:", json_data)
        response_content = normalize_to_queries(response_content)
        queries = Queries.model_validate(json_data)

    except Exception as e:
        print("Error parsing JSON from model output:", e)
        return {"content": state.get('content', [])}

    content = state.get('content', [])
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        print("tavily search response:", response)
        for r in response.get('results', []):
            content_piece = r.get("content", "")
            content.append(str(content_piece))
    print("exiting research_plan_node")
    return {"content": content}
def generation_node(state: AgentState):
    content = "\n\n".join(["content"] or [])
    user_message = HumanMessage(content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message,
    ]
    response = model.invoke(messages)
    response_content = response.content if hasattr(response, 'content') else str(response)

    # Remove <think>...</think> blocks completely
    response_content = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()
    return {
        "draft": response_content,
        "revision_number": state.get("revision_number", 1) + 1,
    }
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state['draft']),
    ]
    response = model.invoke(messages)
    response_content = response.content if hasattr(response, 'content') else str(response)

    # Remove <think>...</think> blocks completely
    response_content = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()

    return {"critique": response_content}

def research_critique_node(state: AgentState):
    try:
        # Run the model
        raw_response = model.invoke([
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
            HumanMessage(content=state['critique'])
        ])
        # Safely get the content
        response_content = raw_response.content if hasattr(raw_response, 'content') else str(raw_response)
        

        # Remove <think>...</think> blocks completely
        response_content = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL)

        # Extract JSON from model output (if Qwen adds extra tokens)
        response_content = normalize_to_queries(response_content)
        queries = Queries.model_validate(response_content)

    except Exception as e:
        print("Error during model invocation or JSON parsing:", e)
        return {"content": state.get("content", [])}

    content = state.get("content", [])
    for q in queries.queries:
        try:
            response = tavily.search(query=q, max_results=2)
            print("tavily response" + response)
            for r in response.get('results', []):
                content_piece = r.get("content", "")
                content.append(str(content_piece))
        except Exception as e:
            print(f"Search failed for query '{q}': {e}")
    
    return {"content": content}

def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"


#build graph
builder = StateGraph(AgentState)
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

#set entry point
builder.set_entry_point("planner")

#define conditional edges
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)

#define edges
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")



#print graph
#Image(graph.get_graph().draw_png())

thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    graph = builder.compile(checkpointer=checkpointer)
    for s in graph.stream({
        'task': "what is the difference between langchain and langsmith",
        "max_revisions": 2,
        "revision_number": 1,
    }, thread):
        print(s)

{'planner': {'plan': '### **Essay Outline: The Difference Between LangChain and LangSmith**\n\n---\n\n#### **I. Introduction**  \n**Purpose:** Introduce the growing importance of large language models (LLMs) and the tools that support their development and deployment.  \n**Key Points:**  \n- Briefly define LLMs and their applications (e.g., chatbots, data analysis).  \n- Mention that tools like **LangChain** and **LangSmith** are critical for building and managing AI workflows.  \n- **Thesis:** While both tools are designed for working with LLMs, they differ in focus, functionality, and target users.  \n\n**Notes for Writer:**  \n- Start with a hook (e.g., "In the era of AI, the tools that power language models are as vital as the models themselves").  \n- Define LLMs briefly to set context.  \n\n---\n\n#### **II. What is LangChain?**  \n**Purpose:** Explain LangChain’s role as a framework for building applications with LLMs.  \n**Key Points:**  \n- **Focus:** Development of AI workflo

In [2]:
from helper import ewriter, writer_gui
MultiAgent = ewriter()
app = writer_gui(MultiAgent.graph)
app.launch(share=True)

* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://34c82e2db1fa16a1b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
